In [26]:
from selenium import webdriver
import tkinter as tk
import time
from tkinter import *
import time
from datetime import datetime
from datetime import date
import textwrap
from random import randint
import json
import string
from tkinter import ttk
import numpy as np
from udpy import UrbanClient
from wiktionaryparser import WiktionaryParser
from nltk.corpus import wordnet as wn
from profanity_filter import ProfanityFilter


In [27]:
from __future__ import division
from nltk.corpus import wordnet as wn
from collections import Counter
import re, nltk
import operator
import urllib, json
from time import sleep
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [28]:
WORDS = nltk.corpus.brown.words()
COUNTS = Counter(WORDS)

def pdist(counter):
    N = sum(counter.values())
    return lambda x: counter[x]/N

P = pdist(COUNTS)

def Pwords(words):
    #"Probability of words, assuming each word is independent of others."
    return product(P(w) for w in words)

def product(nums):
    #"Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def splits(text, start=0, L=20):
   # "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) for i in range(start, min(len(text), L)+1)]

def segment(text):
    #"Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)
    
def segment_decision(word):
    segments = segment(word)
    count = sum(map(lambda x : len(x) <= 1, segments))
    if count > 2 or len(segments) == 1:
        return -1
    result = segments[0]
    for i in range(1, len(segments)):
        result = result + " " + segments[i]
    return result

def parseDictionaries(w):
    parser = WiktionaryParser()
    clues_down_new = []
    clues_acc_new = []
    
    index = w[0]
    print(w)
    word = parser.fetch(w[1:])
    print('the word which will be evaluated: ', w[1:])

    a = wordnet_definition(w[1:])
    if(a != -1):
        clues_down_new.append((int(index),a[0]))
        print('wordnet = ' , a[0])
        return ((int(index), a[0]))
    else:
        try:
            print('Searching wiktionary for ', w[1:])
            print('wiktionary = ' , word[0]['definitions'][0]['text'][1])
            pr = ProfanityFilter()
            if(w[1:] in word[0]['definitions'][0]['text'][1] and not pr.is_profane(word[0]['definitions'][0]['text'][1])):

                return ((int(index), word[0]['definitions'][0]['text'][1]))
            else:
                print('Wiktionary could not find')
                print(word[1249817242839512]['definitions'][0]['text'][1])
        except IndexError:
            googdic = google_dictionary(w[1:])
            if ( googdic != -1):
                clues_down_new.append((int(index), googdic[0]))
                print('google dictionary = ', googdic[0])
                return ((int(index), googdic[0]))
            else:
                try:
                    print('Google dictionary could not find')
                    print('Urban dictionary search started')
                    client = UrbanClient()
                    defs = client.get_definition(w[1:])
                    pr = ProfanityFilter()
                    for i in range(len(defs)):
                        if(pr.is_profane(defs[i].definition)):
                            continue
                        if len(defs[i].definition) > 100:
                            continue
                        if( w[1:].lower() in defs[i].definition.lower()):
                            print('Replacing the word with blank in definition')
                            tmp = defs[i].definition.split('.')[0].lower()
                            tmp = tmp.replace(w[1:].lower(), '______')
                            if ('(1)' in tmp):
                                tmp = tmp[4:]
                            clues_down_new.append((int(index), tmp))
                            print('urban dictionary = ', tmp)
                            return ((int(index), tmp))
                        else:
                            clues_down_new.append((int(index), defs[i].definition.split('.')[0]))
                            print('urban dictionary = ', defs[i].definition.split('.')[0])
                            return ((int(index), defs[i].definition.split('.')[0]))
                except IndexError:
                    return -1
                
def google_dictionary(word):
    #word = word.replace(" ", "%20")
    print("google dictionary search has started")
    word = word.lower()
    try:
        with urllib.request.urlopen("https://api.dictionaryapi.dev/api/v1/entries/en/" + word ) as url:
            print("Taking meanings and synonyms of the word")
            data = json.loads(url.read().decode())
            meaning_dict = data[0]['meaning']
            defs = []
            synonyms = []
            print("if the definition does not include the word or word's root itself, add the definition pool")
            print("Possible synonyms are added synonyms pool")
            for i in meaning_dict.keys():
                for index in range(len(meaning_dict[i])):
                    for key in meaning_dict[i][index].keys():
                        if key == 'definition' and word.lower() not in meaning_dict[i][index][key].lower():
                            defs.append(meaning_dict[i][index][key])
                        if key == 'synonyms':
                            synonyms.extend(meaning_dict[i][index][key])
            synsets = wn.synsets(word)
            syn = []
            if len(synsets) > 0:
                print("Evaluate all the synonyms with respect to path similarity between the original word and the synonyms")
                for i in range(len(synonyms)):
                    if len(wn.synsets(synonyms[i])) > 0 and synonyms[i] != word:
                        val = synsets[0].path_similarity(wn.synsets(synonyms[i])[0])
                        if val != None:
                            syn.append((synonyms[i], val))
                syn.sort(key=operator.itemgetter(1), reverse=True)
            print("sort the definition pool with respect to string length take the smallest string length")
            defs.sort(key=len)
            definition = defs[0]
            if definition[-1:] == '.':
                definition = definition[:-1]
            synonym = ''
            
            if len(syn) > 0:
                synonym = syn[0][0]
            print("return possible definition and the best synonym w.r.t. evaluation above")
            print("google dictionary search has ended")
            return ((definition, synonym))
    except:
        return -1
    
def pdist(counter):
    N = sum(counter.values())
    return lambda x: counter[x]/N

P = pdist(COUNTS)

def Pwords(words):
    #"Probability of words, assuming each word is independent of others."
    return product(P(w) for w in words)

def product(nums):
    #"Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def splits(text, start=0, L=20):
   # "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) for i in range(start, min(len(text), L)+1)]

def segment(text):
    #"Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)
    
def segment_decision(word):
    print("Try to segment to word into the two word, if it is possible")
    segments = segment(word)
    count = sum(map(lambda x : len(x) <= 1, segments))
    if count > 2 or len(segments) == 1:
        return -1
    result = segments[0]
    for i in range(1, len(segments)):
        result = result + " " + segments[i]
    return result
WORDS = nltk.corpus.brown.words()
COUNTS = Counter(WORDS)

def wordnet_definition(word):
    print("wordnet search started")
    word = word.lower()
    ex = []
    non_ex = []
    root = wn.morphy(word)
    print( "finding the root of the word, if it is possible")
    words = wn.synsets(word)
    print( "Taking synset of the word")
    print("Decision Making: If definition has the word itself or its root, do not consider as a clue!")
    print("If a definition has an example, give it a priority.")
    for i in range(len(words)):
        if word in words[i].definition() or (root is not None and root in words[i].definition()):
            continue
        #print(words[i].examples())
        #print(words[i].definition())
        #print("#################################")
        
        if len(words[i].examples()) > 0:
            ex.append(words[i].definition())
        else:
            non_ex.append(words[i].definition())
    print("Sort all the definitions with respect to string length and return the smallest one, with respect to priority given above")
    ex.sort(key = len)
    non_ex.sort(key = len)
    
    print("wordnet search ended")
    if len(ex) > 0:
        return (ex[0], True)
    elif len(non_ex) > 0:
        return (non_ex[0], False)
    else:
        return -1
    
    
def append_clues(data, index, down):
    clues = []
    key = 'Down'
    if down == False:
        key = 'Across'
    for i in range(len(data[index]['clues'][key])):
        clues.append(data[index]['clues'][key][i]['key'])
        clues.append(data[index]['clues'][key][i]['hint'])
    return clues


def append_table(data, index):
    table = []
    for i in range(len(data[index]['solution_cells'])):
        inner_table = []
        if data[index]['solution_cells'][i]['color'] == 'black':
            inner_table.append('')
        elif data[index]['solution_cells'][i]['key'] != '':
            inner_table.append(data[index]['solution_cells'][i]['key'])
            inner_table.append(data[index]['solution_cells'][i]['solution'])
        else:
            inner_table.append(data[index]['solution_cells'][i]['solution'])
        table.append(inner_table)
    return table

def append_solutions(data, index, down):
    sol = []
    key = 'Down'
    if down == False:
        key = 'Across'
    for i in data[index]['solutions'][key].keys():
        string = i + data[index]['solutions'][key][i]
        sol.append(string)
    return sol    

In [4]:
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
##--------------------------------------------------START OF THE SELENIUM BLOCK---------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#

In [ ]:
#--------------------------------------------------------------------------------------#
#GET TODAY'S PUZZLE
nltk.download('wordnet')
driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.nytimes.com/crosswords/game/mini')
time.sleep(3)
#OK BUTTON
driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/main/div[2]/div/div[2]/div[3]/div/article/div[2]/button').click()
time.sleep(3)
#REVEAL BUTTON
driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/main/div[2]/div/div/ul/div[2]/li[2]/button').click()
time.sleep(3)
#REVEAL PUZZLE BUTTON
driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/main/div[2]/div/div/ul/div[2]/li[2]/ul/li[3]/a').click()
time.sleep(3)
#SURE REVEAL BUTTON
driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/article/div[2]/button[2]').click()
time.sleep(3)
#SONDAKi cARPI
driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/span').click()

clues_across = driver.find_elements_by_xpath('//*[@id="root"]/div/div/div[4]/div/main/div[2]/div/article/section[2]/div[1]/ol')[0].text.split('\n')
clues_down = driver.find_elements_by_xpath('//*[@id="root"]/div/div/div[4]/div/main/div[2]/div/article/section[2]/div[2]/ol')[0].text.split('\n')

table = []
for i in range(25):
    css_link = '#xwd-board > g:nth-child(5) > g:nth-child('+str(i+1)+')'
    #print(css_link)
    #print(driver.find_elements_by_css_selector(css_link)[0].text)
    #print('-----------------')
    if not driver.find_elements_by_css_selector(css_link):
        table.append('')
    else:
        table.append(driver.find_elements_by_css_selector(css_link)[0].text.split("\n"))

print('Clues down: ', clues_down)
print('Clues across: ', clues_across)
#--------------------------------------------------------------------------------------#

In [ ]:
#--------------------------------------------------------------------------------------#
#CREATE SOLUTION WORDS

answers_down = []
temp =[]
answers_across = []
for i,w in enumerate(table):
    if w[0].isdigit():
        temp.append((w[0],w[1]))
    else:
        temp.append(w[0])
solutions = np.array(temp).reshape((5,5))
soldown = solutions.T
sol_down = []
sol_acc = []

for i in range(5):
    tmp = ''
    tmp2 = ''
    for j in range(5):
        if(type(soldown[i][j]) is tuple):
            try:
                if tmp[0].isdigit():
                    tmp += soldown[i][j][1]
                else:
                    tmp += soldown[i][j][0] + soldown[i][j][1]
            except IndexError:
                tmp += soldown[i][j][0] + soldown[i][j][1]
        else:
            tmp += soldown[i][j]
    sol_down.append(tmp)
    for j in range(5):
        if(type(solutions[i][j]) is tuple):
            try:
                if tmp2[0].isdigit() and j > 0:
                    tmp2 += solutions[i][j][1]
                else:
                    tmp2 += solutions[i][j][0] + solutions[i][j][1]
            except IndexError:
                tmp2 += solutions[i][j][0] + solutions[i][j][1]
        else:
            tmp2 += solutions[i][j]
    sol_acc.append(tmp2)
#--------------------------------------------------------------------------------------#

In [ ]:
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
##-------------------------------------------END OF THE SELENIUM BLOCK---------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#

In [ ]:
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
##-------------------------------------------START OF THE DATA BLOCK-----------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------------------#

In [29]:
#------------------------------------------data block----------------------------------------------------------------------------
import json
path = "Data/data.json"
index = 131
f = open(path)
m = json.load(f)
m[index]

{'cells': [{'x': 1, 'y': 1, 'color': 'black', 'key': ''},
  {'x': 2, 'y': 1, 'color': 'none', 'key': '1'},
  {'x': 3, 'y': 1, 'color': 'none', 'key': '2'},
  {'x': 4, 'y': 1, 'color': 'none', 'key': '3'},
  {'x': 5, 'y': 1, 'color': 'none', 'key': '4'},
  {'x': 1, 'y': 2, 'color': 'none', 'key': '5'},
  {'x': 2, 'y': 2, 'color': 'none', 'key': ''},
  {'x': 3, 'y': 2, 'color': 'none', 'key': ''},
  {'x': 4, 'y': 2, 'color': 'none', 'key': ''},
  {'x': 5, 'y': 2, 'color': 'none', 'key': ''},
  {'x': 1, 'y': 3, 'color': 'none', 'key': '6'},
  {'x': 2, 'y': 3, 'color': 'none', 'key': ''},
  {'x': 3, 'y': 3, 'color': 'none', 'key': ''},
  {'x': 4, 'y': 3, 'color': 'none', 'key': ''},
  {'x': 5, 'y': 3, 'color': 'none', 'key': ''},
  {'x': 1, 'y': 4, 'color': 'none', 'key': '7'},
  {'x': 2, 'y': 4, 'color': 'none', 'key': ''},
  {'x': 3, 'y': 4, 'color': 'none', 'key': ''},
  {'x': 4, 'y': 4, 'color': 'none', 'key': ''},
  {'x': 5, 'y': 4, 'color': 'none', 'key': ''},
  {'x': 1, 'y': 5, 'col

In [30]:
def data_prep(path, index):
    f = open(path)
    data = json.load(f)
    clues_down = append_clues(m,index, True)
    clues_across = append_clues(m,index, False)
    table = append_table(m, index)
    sol_down = append_solutions(m, index, True)
    sol_acc = append_solutions(m, index, False)
    return (clues_down, clues_across, table, sol_down, sol_acc)

In [31]:
(clues_down, clues_across, table, sol_down, sol_acc) = data_prep(path, index)

In [32]:

#PARSE DICTIONARIES
clues_down_new = []
clues_acc_new = []
for i, w in enumerate(sol_down):
    a = parseDictionaries(w)
    if(a == -1):
        segmented_word = segment_decision(w[1:])
        if segmented_word == -1:
            continue
        segmented_word = str(w[0]) + segmented_word
        a = parseDictionaries(segmented_word)
    clues_down_new.append((a[0], a[1].capitalize()))
for i,w in enumerate(sol_acc):
    a = parseDictionaries(w)
    if(a == -1):
        segmented_word = segment_decision(w[1:]) 
        if segmented_word == -1:
            continue
        segmented_word = str(w[0]) + segmented_word
        a = parseDictionaries(segmented_word)
    clues_acc_new.append((a[0], a[1].capitalize()))
clues_down_new = sorted(clues_down_new)
clues_acc_new = sorted(clues_acc_new)
#--------------------------------------------------------------------------------------#
#GRAPHICAL USER INTERFACE

win = tk.Tk()

win.title("Group: ING")

label = Label(win, text="Original Across", font = "Times 15 bold")
label.grid(row = 0, column = 6)
i = 0
row_i = 0
d = 0
while(i < len(clues_across) + len(clues_acc_new) + 1):
    if (i <  len(clues_across)):
        wrapper = textwrap.TextWrapper(width = 65)
        q = clues_across[i] + ". " + clues_across[(i+1)]
        
        q = wrapper.fill(text = q)
        label = Label(win, text=q, font='Times 12', justify=RIGHT)
        label.grid(row=1 + row_i, column=6, sticky="NW", padx=20, pady=4)
        i += 2
        d += 1
        row_i += 1
    elif(i == len(clues_across)):
        q = 'New Across'
        q = wrapper.fill(text = q)
        label = Label(win, text=q, font='Times 15 bold', justify=LEFT)
        label.grid(row=1 + row_i, column=6, sticky="NW", padx=20, pady=4)
        i += 1 
        row_i += 1
    else:
        wrapper = textwrap.TextWrapper(width = 65)
        q = str(clues_acc_new[i-1-len(clues_across)][0]) + ". " + clues_acc_new[i-1 - len(clues_across)][1]
        if not q[3].isupper():
            if ')' in q:
                index = q.index(')')
                tmp = q[0:index+2]
                tmp2 = q[index+2:len(q)]
                tmp2 = tmp2.capitalize()
                print('bak buraya', tmp2)
                q = tmp + tmp2
        q = wrapper.fill(text = q)
        label = Label(win, text=q, font='Times 12', justify=LEFT)
        label.grid(row=1 + row_i, column=6, sticky="NW", padx=20, pady=5)
        i += 1 
        row_i += 1


label = Label(win, text = "Original Down ", font = "Times 15 bold")
label.grid(row =0, column=7)
i = 0
row_i = 0
while(i < len(clues_down) + len(clues_down_new) + 1):
    if (i < len(clues_down)):
        wrapper = textwrap.TextWrapper(width = 60)
        q = clues_down[i] + ". " + clues_down[i+1]
        
        q = wrapper.fill(text = q)
        label = Label(win, text=q, font='Times 12', justify=LEFT)
        label.grid(row=1 + row_i, column=7, sticky="NW", padx=20, pady=4)
        i += 2 
        row_i += 1
    elif(i == len(clues_down)):
        q = 'New Down'
        q = wrapper.fill(text = q)
        label = Label(win, text=q, font='Times 15 bold', justify=LEFT)
        label.grid(row=1 + row_i, column=7, sticky="NW", padx=20, pady=4)
        i += 1 
        row_i += 1
    else:
        wrapper = textwrap.TextWrapper(width = 60)
        q = str(clues_down_new[i- 1 - len(clues_down)][0]) + ". " + clues_down_new[i - 1 - len(clues_down)][1]
        if not q[3].isupper():
            if ')' in q:
                index = q.index(')')
                tmp = q[0:index+2]
                tmp2 = q[index+2:len(q)]
                tmp2 = tmp2.capitalize()
                print('bak buraya', tmp2)
                q = tmp + tmp2
        q = wrapper.fill(text = q)
        label = Label(win, text=q, font='Times 12', justify=LEFT)
        label.grid(row=1 + row_i, column=7, sticky="NW", padx=20, pady=5)
        i += 1
        row_i += 1
        

cellSize = 75
print('Solutions = ', table)
for i in range(len(table)):
    frame = Frame(win, width=cellSize, height=cellSize)  # their units in pixels
    if table[i][0] == '':
        button = Button(frame,relief="solid",text = '', font = 'Times 33 bold', bg = 'black', disabledforeground = 'blue',state=DISABLED)      
    elif table[i][0].isdigit():
        label = Label(win, text = table[i][0], bg = 'white', font = 'Times 11 bold')
        label.place( x = 5 + int(i % 5) * cellSize, y = 40 + int(i / 5) * cellSize, in_=win)
        button = Button(frame, relief="solid", text=table[i][1], font='Times 30 bold', bg='white',
                    disabledforeground='blue', state=DISABLED)
    else:
        button = Button(frame, relief="solid", text=table[i][0], font='Times 30 bold', bg='white',
                    disabledforeground='blue',fg = 'blue', state=DISABLED)
    
    frame.grid_propagate(False)  # disables resizing of frame
    frame.columnconfigure(0, weight=1)  # enables button to fill frame
    frame.rowconfigure(0, weight=1)  # any positive number would do the trick

    frame.grid(row=int(i / 5) + 1, column=int(i % 5))
    button.grid(sticky="wens")  # makes the button expand
a = datetime.now()
footer = "ING  -  " + a.strftime("%B %d, %Y %H:%M")
#footer = "ING  -  " + date.today().strftime("%B %d, %Y %H %m, %S") 
label = Label(win, text=footer, font='Times 14 bold')
label.grid(row = 6, column = 0, columnspan = 5, sticky = 'E')

win.mainloop()

5SILK
the word which will be evaluated:  SILK
wordnet search started
finding the root of the word, if it is possible
Taking synset of the word
Decision Making: If definition has the word itself or its root, do not consider as a clue!
If a definition has an example, give it a priority.
Sort all the definitions with respect to string length and return the smallest one, with respect to priority given above
wordnet search ended
wordnet =  a fabric made from the fine threads produced by certain insect larvae
1JULIA
the word which will be evaluated:  JULIA
wordnet search started
finding the root of the word, if it is possible
Taking synset of the word
Decision Making: If definition has the word itself or its root, do not consider as a clue!
If a definition has an example, give it a priority.
Sort all the definitions with respect to string length and return the smallest one, with respect to priority given above
wordnet search ended
Searching wiktionary for  JULIA
google dictionary search has 